# Analys av tweets från bokmässan

## Attribution David Johnsson, Uppsala University


Starta med att ladda in följande moduler och sätt upp visualiseringsmiljön för matplotlib

1. `pandas` 
2. `textmining` 
Funktioner för statistisk textmining, fokuserad på bag-of-words model (som ni inte behöver sätta er in för denna kurs.f För den nyfikne eller vetgirige finns enkla förklaringar exempelvis [här](https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/) eller [här](https://www.geeksforgeeks.org/bag-of-words-bow-model-in-nlp/), en enkel tutorial finns också [här](https://machinelearningmastery.com/gentle-introduction-bag-words-model/)) 
3. `wordcloud` - En visualiseringsmodul för att skapa ordmoln, vilket vi gör i denna laboration.
4. `matplotlib` 
5. `sklearn` -  Scikit-learn,ett pythonbibliotek för maskininlärningsalgoritmer, den kommer vi använda mycket i både laboration 3 och 4.

In [ ]:
# Kör denna cell för att ladda in biblioteken och sätta upp vår miljö
import itertools

import matplotlib
import nltk
import pandas as pd
import textmining as tm
import wordcloud
from sklearn.feature_extraction.text import CountVectorizer

# Sätt upp visualiseringen
%matplotlib inline
matplotlib.pyplot.rcParams["figure.figsize"] = [10, 6]

## Analys av Twitterdata från bokmässan

Ni har blivit inhyrda som konsulter för en bokpublicist som vill att du ska ta reda på vilka teman och böcker som har fått mest uppmärksamhet på bokmässan i Göteborg 2016. 

Er uppgift är att via Twitterdata undersöka vilka ämnen som fått speciellt mycket uppmärksamhet för och under bokmässan och presentera ett förslag till företaget du arbetar med vad som är lämpliga debattämnen. 

Fokus här är alltså på att förstå data, vilket är en viktigt del av pre-processering inför mer avacerad dataanalys. 


## Data processing

Som alltid behöver vårt data städas, i detta fall är fokus att sortera bort data som antingen inte går att analysera eller inte är intressant från den råtextdata vi fått från Twitter. Den data som givits samlades in från Twitter från maj till september 2016.

Er datafil finns i repositoriet på github  och heter `twitter_book_fair_data.tsv`.

### Ladda data

En `.tsv` fil betyder att det är en tab-separerad fil med tabelldata (jämfört med ; separerad som vi använt tidigare)

**F2** Starta arbetet med att läsa in filen med read_csv() med följande parametrar:  encoding="utf-8", sep="\t" och spara i en dataframe

In [ ]:
twitter_data_book_df = pd.read_csv(
    "Data/twitter_book_fair_data.tsv", encoding="utf-8", sep="\t"
)

In [ ]:
twitter_data_book_df.head()

In [ ]:
twitter_data_book_df.shape

In [ ]:
twitter_data_book_df.dtypes

In [ ]:
twitter_data_book_df.isnull()

In [ ]:
twitter_data_book_df.info()

En kolumn är speciellt intressant för vår **textanalys**, extrahera den från den dataframe vi lagrat all data i och skapa en variabel där du placerar denna data, döp variablen till `tweets_corpus`.

In [ ]:
tweets_corpus = twitter_data_book_df.text

In [ ]:
tweets_corpus.head()

### Emojis

På Twitter är det väldigt vanligt med emojis 👍 ✨ 🐫 🎉 🚀 🤘.

Dessa kan innehålla mycket information som kan vara relevant för vår analys. Dock är det ofta svårt att analysera emojis med hjälp av vanliga verktug för NLP(Natural Language Processig). 

Vi behöver därför ta bort dessa ur vårt utvalda dataset som skapades i uppgiften ovan.


In [ ]:
encode2ascii = lambda x: x.encode("ascii", errors="ignore").decode("utf-8")
clean_tweets = tweets_corpus.apply(encode2ascii)
clean_tweets

### Ta bort URLs
Det är också vanligt att man på Twitter länkar till olika webbplatser med hjälp av URL:er, när man gör textanalys på twitterdata är det vanligt att delar av dessa URL:er dyker upp som "mest frekventa ord" vilket påverkar vår analys negativs. Dessa behöver därför också tas bort.

In [ ]:
clean_tweets = clean_tweets.str.replace(r"http\S+", "")
clean_tweets

### Funktion för att hitta mest frekventa ord 

Ett sätt att förstå hur olika metoder för pre-processing påverkar ett dataset kan man räkna de mest förekommande orden efter varje operation som utförs. Eftersom vi kommer vilja utföra denna räkning många gånger under arbetet är de lämpligt att skapa en funktion för det som vi kan anropa flera gånger.

#### Vad är en Term Document Matrix (TDM)?

En TDM är en tabell där antalet unika ord räknas för varje dokument. För att göra detta på vårt Twitterdata är det lämpligt att skapa en TDM där varje tweet är en egen vektor där varje element består av de ord som finns i den tweeten. En tweet med tre unika ord blir alltså en vektor med tre element. 

Nedanstående kod skapar denna TDM i form av en funktion med namn `create_term_document_matrix()`:

In [ ]:
"""
Skapar en funktion som tar in argumenten corpus och min_df, om inget värde 
ges till min_df så blir det 1
"""


def create_term_document_matrix(corpus, min_df=1):
    cvec = CountVectorizer(min_df=min_df, stop_words=tm.stopwords)
    """
    CountVectorizer skapar en matris med varje unikt ord i en datasamling
    och visar hur ofta de förekommer. Argumentet stop_words ger användaren
    möjligheten att välja vilka ord som ska vara stopwords. I detta fall
    används textminings fördefinierade stop words. min_df definierar den undre
    gränsen för hur ofta ett ord behöver förekomma för att vara med i matrisen
    """

    tfmatrix = cvec.fit_transform(corpus)
    # Transformerar en datasamling tll valfri struktur genom sitt agrument
    return pd.DataFrame(data=tfmatrix.toarray(), columns=cvec.get_feature_names())
    """
    Funktionen returnerar en dataframe där datamängden kommer från tfmatrix och 
    kolumn namnen från cvec. I det här fallet är kolumnnamnen de unika orden. 
    Funktionen är alltså en implementation av bag-of-words, där varje förekomst av varje unikt ord (undantaget stopwords) räknas.
    """

Testa vår nya funktion genom att skapa en TDM endast för de tre första raderna i `clean_tweets` som kan sorteras ut med `.head(3)` funktionen. 

In [ ]:
create_term_document_matrix(clean_tweets.head(3))

**F11.** Hur många kolumner skapades i TDM:n?

Svaret beror lite på vilka parametrar man lägger in i funktionen ovan, men man bör kunna plocka fram antalet med exempelvis shape och också förhoppningsvis förstå att antalet kolumner representerar antalet unika ord vars förekomst räknas. 

För att hitta de mest frekvent förekommander orden i vår TDM behöver vi räkna ord. Det är också lämpligt med en visualisering över dessa vanligast förekommande ord. Även detta kommer vi behöva göra flera gånger och därför är det återigen lämpligt att definiera en funktion `plot_top_words()` som både räknar och plottar orden i ett stapeldiagram. 


In [ ]:
"""
En funktion som tar in en datasamling, antalet ord
som ska finnas i den lista över top words som returnerar samt antalet ord som ska visas i stapeldiagramet. 
Från början ska listan innehålla 50 ord och diagrammet 30. Det finns dock de som ändrat detta när de svarar. 
"""


def plot_top_words(tweets, num_word_instances, top_words):
    tdm_df = create_term_document_matrix(tweets, min_df=2)
    # Använder den tidigare sakpta funktionen för att ta fram en matris med ord
    # som förekommer minst 2 ggr
    word_frequencies = tdm_df[[x for x in tdm_df.columns if len(x) > 1]].sum()
    # Använder en loop för att se om ordet i kolumnen innehåller fler än en
    # bokstav och sumerar sedan kolumnen
    sorted_words = word_frequencies.sort_values(ascending=False)
    # Vi sorterar sedan orden i fallande ordning d.v.s.
    # den mest förekommande först
    top_sorted_words = sorted_words[:num_word_instances]
    # top_sorted_words är sedan de num_word_instances mest förekommande
    # orden från sorted_words
    top_sorted_words[:top_words].plot.bar()
    # Sedan förkortas listan ytterligare med top_words
    # som därefter ritas upp i ett stapeldiagram
    return top_sorted_words
    # top_sorted_words med num_word_instances antalet ord returneras som en lista

Nu kan vi använda `plot_top_words()` funktionen för att räkna ut de mest förekommande orden i hela vårt corpus, viktigt att ha tålamod dock för det kan ta ett tag. Nedanstående kod utför beräkningen.

In [ ]:
top_words = plot_top_words(clean_tweets, 50, 30)
top_words

### Små bokstäver

Nästa steg i pre-processingen av vårt dataset (vårt corpus) är att göra om alla bokstäver till små. 


In [1]:
tweets_lowered = clean_tweets.str.lower()

NameError: name 'clean_tweets' is not defined

In [ ]:

top_words_lowered = plot_top_words(tweets_lowered, 50, 30)
top_words_lowered

För att underlätta att jämföra vad våra ansträngningar får för resultat kan det vara bra att enkelt kunna jämföra olika listor med top_words.

Skapa en ny dataframe som har två kolumner, en med de 20 mest frekventa orden från`top_words` och en med de 20 mest frekventa orden från `top_word_lowered`. Döp kolumnerna till `Top tweeted clean`och  `Top tweeted lowered`. 

In [ ]:
pd.DataFrame(
    {
        "Top tweeted clean": top_words[0:20].index,
        "Top tweeted lowered": top_words_lowered[0:20].index,
    }
)

In [ ]:
set(top_words[0:20].index) - set(top_words_lowered[0:20].index)

### Korta ord

Korta ord har ofta inte någon egentlig betydelse, alltså behöver vi inte dessa ord. Typiska sådana ord kan vara ja, jo eller nej. Vi bestämmer oss för att alla ord som är kortare än 3 bokstäver inte innehar någon betydelse i vår analys och tar därmed bort dem. 


In [ ]:
tweets_lowered

In [ ]:
tweets_low_no_small = tweets_lowered.str.replace(r"\b\w{1,2}\b", "")#Jag brukar ge godkänt om man fyllt i 2 eller 3 här, kommer inte riktigt ihåg vilket som är rätt.
tweets_low_no_small

In [ ]:
# Skapar ny topplista utan korta ord
top_words_low_no_small = plot_top_words(tweets_low_no_small, 50, 30)
top_words_low_no_small

**F21.** Efter att korta ord tagits bort, hur många gånger måste ett ord förekomma i vårt corpus för att hamna i den nya listan enligt ovan? 

### Betydelselösa ord

Stop words är andra ord som inte är korta men som ändå inte har betydelse, dessa kan vara lite besvärligare att identifiera och ta bort. En möjlighet är att helt enkelt skapa en lista med sådana ord och sedan använda den listan för att filtrera ut orden ur ett corpus. Vi har ju redan tagit bort alla ord med färre bokstäver än 3, så sådana behöver vi inte lägga in i listan. 

Nedan är ett exempel på en lista med stoppord som är betydelselösa. 


In [ ]:
my_stop_words = [
    "och",
    "det",
    "att",
    "i",
    "en",
    "jag",
    "hon",
    "som",
    "han",
    "paa",
    "den",
    "med",
    "var",
    "sig",
    "foer",
    "saa",
    "till",
    "aer",
    "men",
    "ett",
    "om",
    "hade",
    "de",
    "av",
    "icke",
    "mig",
    "du",
    "henne",
    "daa",
    "sin",
    "nu",
    "har",
    "inte",
    "hans",
    "honom",
    "skulle",
    "hennes",
    "daer",
    "min",
    "man",
    "ej",
    "vid",
    "kunde",
    "naagot",
    "fraan",
    "ut",
    "naer",
    "efter",
    "upp",
    "vi",
    "dem",
    "vara",
    "vad",
    "oever",
    "aen",
    "dig",
    "kan",
    "sina",
    "haer",
    "ha",
    "mot",
    "alla",
    "under",
    "naagon",
    "eller",
    "allt",
    "mycket",
    "sedan",
    "ju",
    "denna",
    "sjaelv",
    "detta",
    "aat",
    "utan",
    "varit",
    "hur",
    "ingen",
    "mitt",
    "ni",
    "bli",
    "blev",
    "oss",
    "din",
    "dessa",
    "naagra",
    "deras",
    "blir",
    "mina",
    "samma",
    "vilken",
    "er",
    "saadan",
    "vaar",
    "blivit",
    "dess",
    "inom",
    "mellan",
    "saadant",
    "varfoer",
    "varje",
    "vilka",
    "ditt",
    "vem",
    "vilket",
    "sitta",
    "saadana",
    "vart",
    "dina",
    "vars",
    "vaart",
    "vaara",
    "ert",
    "era",
    "vilka",
]

När vi skapat vår lista är det dags att skapa en funktion som tar bort dessa från ett dokument. Denna funktion är kodad i cellen nedan. (Igen strunta i lambda för tillfället.)

In [ ]:
remove_stopwords = lambda x: " ".join(y for y in x.split() if y not in my_stop_words)

Funktionen ovan tar alltså bort stoppord från ett dokument (alltså en tweet), för att ta bort stoppord från hela vårt corpus kan funktionen `.apply()`användas. 


In [ ]:
tweets_low_no_small_stopwords = tweets_low_no_small.apply(remove_stopwords)

In [ ]:
top_words_low_no_small_stopwords = plot_top_words(tweets_low_no_small_stopwords, 50, 30)
top_words_low_no_small_stopwords

Vad är skillnaderna mellan de frekvent förekommande orden i jämförelse med våra tidigare listor? Skriv den kod som jämför dessa tre listor `top_words_lowered`, `top_words_low_no_small` and `top_words_low_no_small_stopwords`, titta på de första 20 orden i listorna.

En variant är att använda koden nedan som jämför listorna och returnerar skillanderna. En annan är att återanvända koden som skapar en dataframe med de tre listorna. Båda varianterna finns med nedan. Man kan säkert göra på andra sätt också. 


In [ ]:
set(top_words_lowered[0:20].index) - set(top_words_low_no_small_stopwords[0:20].index) - set(top_words_low_no_small[0:20].index)

In [ ]:
pd.DataFrame(
    {
        "Top tweeted clean": top_words[0:20].index,
        "Top tweeted lowered": top_words_lowered[0:20].index,
        "Top tweeted no small": top_words_low_no_small[0:20].index,
    }
)

### Visualisering och rekommendation

Dags att visualisera vårt resultat och övertyga vår klient om att vi hittat de bästa debattämnena för dem! Här gör vi det genom att skapa ett word cloud där de mest frekventa orden syns bäst. 

Nedanstående kod skapar ett ordmoln för `top_words_low_no_small_stopwords`

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

wordcloud = WordCloud(max_font_size=40)
wordcloud.fit_words(top_words_low_no_small_stopwords.to_dict())
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

När du tittar på ordmolnet, är det fler ord som borde vara stoppord? Ange några stycken och förklara varför de bör tas bort.

Vilket tema rekommenderar ni att publicisten ska ha som debattämne? 